In [1]:
import os
import pandas as pd
import numpy as np
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import tensorflow as tf

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
fileObject = open("./documents/comp.windows.x_66950.txt", "r")
data = fileObject.read()

In [ ]:
import regex as re
def preprocess_email(data):
    emails = re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", data)
#     print(emails)
    words= [email.split(sep='@')[1] for email in emails]
    words = [item for i in words for item in i.split(sep='.')]
    words = [item for item in words if len(item) > 2]
    preprocessed_email = [" ".join(words)]
    data = re.sub(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", ' ',data)
    return data, preprocessed_email

In [ ]:
def Subject(data):
    word= re.findall("Subject:.*", data)
    word = [item for item in word[0].split() if ':' not in item]
    word = [re.sub('[^a-zA-Z0-9]+', '', _) for _ in word]
    preprocessed_subject = [" ".join(word)]
    data = re.sub("Subject:.*",' ', data)
    return data, preprocessed_subject

In [ ]:
def remove_write_from(data):
    paragraph = re.sub('From:.*',' ',data)
    paragraph = re.sub('Write to:.*',' ',paragraph)
    return paragraph

In [ ]:
def remove_newline(data):
    word = re.sub('\n',' ',data)
    word = re.sub('\t',' ',word)
    word = re.sub('-', ' ',word)
    word = re.sub('/',' ' ,word)
    word = re.sub(r'\w+:\s?',' ' ,word)
    return word

In [ ]:
def remove_brackets(data):
    paragraph = data.replace("(","<")
    paragraph = paragraph.replace(")",">")
    paragraph = re.sub('<[^>]+>', '', paragraph)
    return paragraph

In [ ]:
def remove_words(data):
    data = data.replace('can\'t','can not')
    data = data.replace('\'s','is')
    data = data.replace('i\'ve','i have')
    data = data.replace('i\'m','i am')
    data = data.replace('you\'re','you are')
    data = data.replace('i\'ll','i will')
    data = data.replace('haven\'t','have not')
    return data
    
    

In [ ]:
def chunking(data):
    chunks = []
    chunks = (list(ne_chunk(pos_tag(word_tokenize(data)))))
    for chunk in chunks:
        if type(chunk) == Tree:
            if chunk.label() == 'GPE':
                if len(chunk.leaves()) > 1:
                    word = "_".join([i for i,j in chunk.leaves()])
                    for i in range(len(chunk.leaves())): 
                        if i< len(chunk.leaves())-1:
                            data = re.sub(chunk.leaves()[i][0],'',data,flags=re.MULTILINE)
                        else:
                            data = re.sub(chunk.leaves()[i][0],word,data,flags=re.MULTILINE)
            if chunk.label() == 'PERSON':
                for term,pog in chunk.leaves():
                    data = re.sub(re.escape(term),"",data, flags=re.MULTILINE)
#                 for i in range(len(chunk.leaves())):
#                     data = re.sub(chunk.leaves()[i][0],'',data,flags=re.MULTILINE)   
    return data

In [ ]:
def remove_digits(data):
    data = re.sub('\d',"",data)
    return data

In [ ]:
def _word_(data):
    data = re.sub(r"\b_([a-zA-z]+)_\b",r"\1",data)
    data = re.sub(r"\b_([a-zA-z]+)\b",r"\1",data)
    data = re.sub(r"\b([a-zA-z]+)_\b",r"\1",data)
    data = re.sub(r"\b[a-zA-Z]{1}_([a-zA-Z]+)",r"\1",data)
    data = re.sub(r"\b[a-zA-Z]{2}_([a-zA-Z]+)",r"\1",data)
    return data

In [ ]:
def lower_alpha(data):
    data.lower()
    data = re.sub(r'\b\w{1,2}\b', '', data)
    data = re.sub(r'\b\w{15,}\b', '', data)
    data = re.sub(r"[^a-zA-Z_]"," ",data) 
    data = re.sub(' +', ' ', data)
    return data

In [ ]:
def preprocess_file(Input_Text):
    """Do all the Preprocessing as shown above and
    return a tuple contain preprocess_email,preprocess_subject,preprocess_text for that Text_data"""
    text, list_of_preprocessed_emails = preprocess_email(Input_Text)
    text, subject = Subject(text)
    text = remove_write_from(text)
    text = remove_newline(text)
    text = remove_brackets(text)
    text = remove_words(text)
    text = chunking(text)
    text = remove_digits(text)
    text = _word_(text)
    text = lower_alpha(text)
    return (list_of_preprocessed_emails,subject,text)

In [ ]:
def preprocess(files):
    emails_preprocessed = []
    sub_preprocessed = []
    text_preprocessed = []
    for file in files:
        list_of_preprocessed_emails,subject,text =preprocess(file)
        emails_preprocessed.append(list_of_preprocessed_emails)
        sub_preprocessed.append(subject)
        text_preprocessed.append(text)
    df = pd.DataFrame(list(zip(text_preprocessed, emails_preprocessed,sub_preprocessed)),columns =['text', 'emails','subject'])
    return df

In [ ]:

print(os.getcwd())
os.chdir(r'C:\Users\divya\PycharmProjects\Applied_ai\doc_classification_cnn\documents')
files =[]
output = []
for file in (os.listdir()):
    output.append(file.split('_')[0])
    fileObject = open(file, "r")
    files.append(fileObject.read())


In [ ]:
# list_of_preprocessed_emails,subject,text = preprocess_file(files[0])

In [ ]:
emails_preprocessed = []
sub_preprocessed = []
text_preprocessed = []
for i in range(len(files)):
    list_of_preprocessed_emails,subject,text =preprocess_file(files[i])
    emails_preprocessed.append(list_of_preprocessed_emails)
    sub_preprocessed.append(subject)
    text_preprocessed.append(text)

In [ ]:
df = pd.DataFrame(list(zip(text_preprocessed, emails_preprocessed,sub_preprocessed,output)),columns =['text', 'emails','subject','label'])


In [ ]:
df.head()

In [ ]:
df['concat_string'] = df['subject'].map(lambda x: x[0])+df['emails'].map(lambda x: x[0])+df['text']


In [ ]:
df.head()

In [ ]:
df.to_csv('doc_data.csv')

In [3]:
os.chdir(r'C:\Users\divya\PycharmProjects\Applied_ai\doc_classification_cnn')
df = pd.read_csv('doc_data.csv')

In [4]:
df.head()

,text,emails,subject,label,concat_string
0,Archive atheism resources Alt atheism archive...,['mantis netcom com mantis'],['AltAtheism Atheist Resources'],alt.atheism,AltAtheism Atheist Resourcesmantis netcom com ...
1,rchive atheism introduction atheism archive i...,['mantis mantis mantis'],['AltAtheism Introduction to Atheism'],alt.atheism,AltAtheism Introduction to Atheismmantis manti...
2,article Well has quite different not necessar...,['dbstu1 tu-bs mimsy umd edu umd edu'],['Gospel Dating'],alt.atheism,Gospel Datingdbstu1 tu-bs mimsy umd edu umd ed...
3,Recently RAs have been ordered apparently pos...,['mantis kepler unh edu'],['university violating separation of churchsta...,alt.atheism,university violating separation of churchstate...
4,article HOWEVER hate economic terrorism and p...,['harder ccr-p ida org harder ccr-p ida org wa...,['socmotss et al Princeton axes matching funds...,alt.atheism,socmotss et al Princeton axes matching funds f...


In [5]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelBinarizer
X = np.array(df['concat_string']) 
Y = to_categorical(np.asarray(df['label'].factorize()[0]))
print(Y.shape)

(18875, 43)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [7]:
MAX_WORDS = 1000
tokenizer  = Tokenizer(num_words = MAX_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
tokenizer.fit_on_texts(X_train.astype(str))
tokenize_X_train =  tokenizer.texts_to_sequences(X_train.astype(str))
tokenize_X_test =  tokenizer.texts_to_sequences(X_test.astype(str))
word_index = tokenizer.word_index

In [8]:
max_padd_len =1000
X_train_pad = pad_sequences(tokenize_X_train, maxlen=max_padd_len)
X_test_pad = pad_sequences(tokenize_X_test, maxlen=max_padd_len)

In [9]:
glove_name= r'C:\Users\divya\PycharmProjects\Applied_ai\doc_classification_cnn/glove.6B.50d.txt'
with open(glove_name,"r",encoding="utf8") as f:
    words= []
    word_to_vec= {}
    for line in f:
        line= line.strip().split()
        word_in_sen= line[0]
        words.append(word_in_sen)
        word_to_vec[word_in_sen]= np.array(line[1:], dtype= 'float64')
print("Number of word to vector conversion:{}".format(len(word_to_vec.keys())))

Number of word to vector conversion:400000


In [10]:
embedding_matrix = np.zeros((len(word_index)+1,50))
for word,i in word_index.items():
    if word_to_vec.get(word) is not None:
        embedding_matrix[i] = word_to_vec[word]
        

In [11]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], 
                            input_length=max_padd_len,
                            trainable=False)

In [12]:
def model_1(input_shape = (max_padd_len,) , classes = 43):
    X_input = Input(input_shape)
    embedding = embedding_layer(X_input)
    embedding = Reshape((max_padd_len,50))(embedding)
    conv_1 = Conv1D(filters=8, kernel_size=3,strides = 4, activation='relu', input_shape=(max_padd_len,100), padding='valid')(embedding)
    conv_2 = Conv1D(filters=8, kernel_size=4,strides = 4, activation='relu', input_shape=(max_padd_len,100), padding='valid')(embedding)
    conv_3 = Conv1D(filters=8, kernel_size=5,strides = 4,activation='relu', input_shape=(max_padd_len,100), padding='valid')(embedding)
    
    concat = Concatenate(axis=1)([conv_1, conv_2, conv_3])
    
    X = MaxPool1D(pool_size=3)(concat)
    
    conv_4 = Conv1D(filters=8, kernel_size=4,strides = 4, activation='relu', padding='valid')(X)
    conv_5 = Conv1D(filters=8, kernel_size=5,strides = 4, activation='relu', padding='valid')(X)
    conv_6 = Conv1D(filters=8, kernel_size=6,strides = 4, activation='relu', padding='valid')(X)
    
    concat = Concatenate(axis=1)([conv_4, conv_5, conv_6])
    
    X = MaxPool1D(pool_size=3)(concat)
    
    X = Conv1D(filters=8, kernel_size=5,strides = 4, activation='relu', padding='valid')(X)
    
    X = Flatten()(X)
    X = Dropout(0.2)(X)
    X = Dense(48, activation='relu')(X)
    X = Dense(classes, activation='softmax')(X)
    model = Model(inputs = X_input, outputs = X)
    return model
    

In [13]:
model = model_1(input_shape = (max_padd_len,), classes = y_train.shape[1])
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1000)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1000, 50)     4106400     ['input_1[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 1000, 50)     0           ['embedding[0][0]']              
                                                                                                  
 conv1d (Conv1D)                (None, 250, 8)       1208        ['reshape[0][0]']                
                                                                                              

In [14]:
import datetime
# Load the TensorBoard notebook extension
%load_ext tensorboard
# Clear any logs from previous runs
# history_own=LossHistory(validation_data=[X_test_pad,y_test])  
filepath="model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')
earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0.35, patience=2, verbose=1)
log_dir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [15]:
import tensorflow_addons as tfa
callbacks_list = [checkpoint,earlystop,tensorboard_callback]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',tfa.metrics.F1Score(num_classes=43, average="micro")])

In [ ]:
model.fit(X_train_pad, y_train,validation_data=(X_test_pad,y_test), epochs=10, batch_size=64,callbacks=callbacks_list)

Epoch 1/10
